In [1]:
%load_ext autoreload

In [2]:
import scanpy as sc
import anndata as ad
import scirpy as ir

import pandas as pd
import numpy as np
from tqdm import tqdm

C:\Users\felix.drost\Anaconda3\envs\covidVac240624\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%autoreload 2
import sys
sys.path.append('..')

import utils.annotation as utils_annotation

In [4]:
path_results = '../../results/dextramer/CD8/revision2'
path_data = '../../data/dextramer/02_dex_annotated_cd8.h5ad'

In [5]:
adata = sc.read(path_data)

In [6]:
binding_mode = 'binding_ct'
time_order = ['P1', 'S1', 'S2', 'S3', 'T1', 'T2', 'T3', 'X3', 'extra']

## Suppl. Fig. 2E

In [7]:
adata_tmp = adata[adata.obs['clone_id']!='nan']
adata_tmp = adata_tmp[adata_tmp.obs[binding_mode]!='No binding']
content = pd.DataFrame(adata_tmp.obs.groupby(['donor', 'time', binding_mode])['clone_id'].nunique())
content = content.reset_index()
content = content.pivot_table(index='donor', columns=['time', binding_mode], values='clone_id')
content.columns = [f'{col[0]}_{col[1]}' if isinstance(col, tuple) else col for col in content.columns]
content.head(3)

,P1_CTELKLSDY,P1_FLRGRAYGL,P1_FPQSAPHGV,P1_KCYGVSPTK,P1_LTDEMIAQY,P1_NYNYLYRLF,P1_QPYRVVVL,P1_QYIKWPWYI,P1_RAKFKQLL,P1_RLQSLQTYV,...,extra_LTDEMIAQY,extra_NYNYLYRLF,extra_QPYRVVVL,extra_QYIKWPWYI,extra_RAKFKQLL,extra_RLQSLQTYV,extra_SPRRARSVA,extra_VLNDILSRL,extra_YLQPRTFLL,extra_YTNSFTRGVY
donor,,,,,,,,,,,,,,,,,,,,,
A02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
content2 = pd.DataFrame(adata_tmp.obs.groupby(['donor', binding_mode])['clone_id'].nunique())
content2 = content2.reset_index()
content2 = content2.pivot_table(index='donor', columns=[binding_mode], values='clone_id')
content2.columns.names = [None]
content2.head(3)

,CTELKLSDY,FLRGRAYGL,FPQSAPHGV,KCYGVSPTK,LTDEMIAQY,NYNYLYRLF,QPYRVVVL,QYIKWPWYI,RAKFKQLL,RLQSLQTYV,SPRRARSVA,VLNDILSRL,YLQPRTFLL,YTNSFTRGVY
donor,,,,,,,,,,,,,,
A02,0,0,0,0,0,0,0,1,0,0,0,0,0,0
A03,0,0,0,32,0,0,0,0,0,1,0,0,6,0
A04,0,0,8,0,22,0,0,0,0,0,20,0,0,2


In [9]:
content3 = pd.DataFrame(adata_tmp.obs.groupby(['donor'])['clone_id'].nunique())
content3.columns = ['#Dex+CTs']
content3.head(3)

,#Dex+CTs
donor,
A02,1
A03,39
A04,52


In [10]:
content4 = pd.DataFrame(adata.obs.groupby(['donor'])['clone_id'].nunique())
content4.columns = ['#CD8-CTs']
content4.head(3)

,#CD8-CTs
donor,
A02,21
A03,161
A04,1659


In [11]:
content5 = pd.DataFrame(adata.obs.groupby(['donor', 'time'])['clone_id'].nunique())
content5 = content5.reset_index()
content5 = content5.pivot_table(index='donor', columns=['time'], values='clone_id')
content5.columns = [f'#CD8-CTs_{t}' for t in content5.columns]
content5.head(3)

,#CD8-CTs_P1,#CD8-CTs_S1,#CD8-CTs_S2,#CD8-CTs_S3,#CD8-CTs_T1,#CD8-CTs_T2,#CD8-CTs_T3,#CD8-CTs_X3,#CD8-CTs_extra
donor,,,,,,,,,
A02,0,0,21,0,0,0,0,0,0
A03,0,0,161,0,0,0,0,0,0
A04,352,263,66,342,604,139,369,0,0


In [12]:
content6 = pd.DataFrame(adata_tmp.obs.groupby(['donor', 'time'])['clone_id'].nunique())
content6 = content6.reset_index()
content6 = content6.pivot_table(index='donor', columns=['time'], values='clone_id')
content6.columns = [f'#Dex+-CTs_{t}' for t in content6.columns]
content6.head(3)

,#Dex+-CTs_P1,#Dex+-CTs_S1,#Dex+-CTs_S2,#Dex+-CTs_S3,#Dex+-CTs_T1,#Dex+-CTs_T2,#Dex+-CTs_T3,#Dex+-CTs_X3,#Dex+-CTs_extra
donor,,,,,,,,,
A02,0,0,1,0,0,0,0,0,0
A03,0,0,39,0,0,0,0,0,0
A04,3,11,9,6,33,13,10,0,0


In [13]:
content_full = pd.concat([content4, content5, content3, content6, content2, content], axis=1)
content_full.to_csv(f'{path_results}/clonotype_counts_specificity.csv')
content_full.head(3)

,#CD8-CTs,#CD8-CTs_P1,#CD8-CTs_S1,#CD8-CTs_S2,#CD8-CTs_S3,#CD8-CTs_T1,#CD8-CTs_T2,#CD8-CTs_T3,#CD8-CTs_X3,#CD8-CTs_extra,...,extra_LTDEMIAQY,extra_NYNYLYRLF,extra_QPYRVVVL,extra_QYIKWPWYI,extra_RAKFKQLL,extra_RLQSLQTYV,extra_SPRRARSVA,extra_VLNDILSRL,extra_YLQPRTFLL,extra_YTNSFTRGVY
donor,,,,,,,,,,,,,,,,,,,,,
A02,21,0,0,21,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A03,161,0,0,161,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A04,1659,352,263,66,342,604,139,369,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
content_full.columns.tolist()

['#CD8-CTs',
 '#CD8-CTs_P1',
 '#CD8-CTs_S1',
 '#CD8-CTs_S2',
 '#CD8-CTs_S3',
 '#CD8-CTs_T1',
 '#CD8-CTs_T2',
 '#CD8-CTs_T3',
 '#CD8-CTs_X3',
 '#CD8-CTs_extra',
 '#Dex+CTs',
 '#Dex+-CTs_P1',
 '#Dex+-CTs_S1',
 '#Dex+-CTs_S2',
 '#Dex+-CTs_S3',
 '#Dex+-CTs_T1',
 '#Dex+-CTs_T2',
 '#Dex+-CTs_T3',
 '#Dex+-CTs_X3',
 '#Dex+-CTs_extra',
 'CTELKLSDY',
 'FLRGRAYGL',
 'FPQSAPHGV',
 'KCYGVSPTK',
 'LTDEMIAQY',
 'NYNYLYRLF',
 'QPYRVVVL',
 'QYIKWPWYI',
 'RAKFKQLL',
 'RLQSLQTYV',
 'SPRRARSVA',
 'VLNDILSRL',
 'YLQPRTFLL',
 'YTNSFTRGVY',
 'P1_CTELKLSDY',
 'P1_FLRGRAYGL',
 'P1_FPQSAPHGV',
 'P1_KCYGVSPTK',
 'P1_LTDEMIAQY',
 'P1_NYNYLYRLF',
 'P1_QPYRVVVL',
 'P1_QYIKWPWYI',
 'P1_RAKFKQLL',
 'P1_RLQSLQTYV',
 'P1_SPRRARSVA',
 'P1_VLNDILSRL',
 'P1_YLQPRTFLL',
 'P1_YTNSFTRGVY',
 'S1_CTELKLSDY',
 'S1_FLRGRAYGL',
 'S1_FPQSAPHGV',
 'S1_KCYGVSPTK',
 'S1_LTDEMIAQY',
 'S1_NYNYLYRLF',
 'S1_QPYRVVVL',
 'S1_QYIKWPWYI',
 'S1_RAKFKQLL',
 'S1_RLQSLQTYV',
 'S1_SPRRARSVA',
 'S1_VLNDILSRL',
 'S1_YLQPRTFLL',
 'S1_YTNSFTRGVY',
 '

In [16]:
content_full['S3_LTDEMIAQY']

donor
A02     0.0
A03     0.0
A04     4.0
A05     0.0
A06     0.0
A07     0.0
A08    14.0
A11     0.0
A15     0.0
A16     0.0
A19     NaN
A25     0.0
A29     0.0
HIM     0.0
Name: S3_LTDEMIAQY, dtype: float64